In [2]:
%load_ext Cython 
import numpy as np
import random, time
from multiprocessing import Pool, freeze_support
from io import StringIO
import itertools,time
import matplotlib.widgets as widgets
%pylab inline
def linlin(x, smi, sma, dmi, dma): return (x-smi)/(sma-smi)*(dma-dmi)+dmi
from threading import Thread

# Audio server
from pyo import *
# Is 1024 too large for the bufferSize in this case. 
fs = 44100/2
s = Server(sr=fs, nchnls=2, buffersize=2048, duplex=0).boot()
s.start()

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
class playAudioNoThread:       
    def __init__(self, sampling_rate, dataLen = 1.0,  play_back_rate = 1.0, bs = 44100, amp = 0.5):
#         audioVec = np.zeros(bufferSize)
        self.sf = sampling_rate
        self.rate = play_back_rate
        self.amp = amp
        self.bs = int(bs)
        self.dataLen = dataLen
        self.t = DataTable(size =self.bs, chnls=1)  # Data table
        self.c = Osc(table=self.t, freq=[ 1.0/self.dataLen * self.rate, \
                                         1.0 / self.dataLen * self.rate],\
                     mul=self.amp).out() # Table playback. 

    def looping(self, audioVec):
        # Continous mode
        self.t.replace(list(audioVec))
        self.c.setTable(self.t)
    
    def single(self, audioVec):
        # Currently no add silence. Need to do so. 
        self.t = DataTable(size =self.bs, chnls=1)  # Data table
        self.c = Osc(table=self.t, freq=[ 1.0/self.dataLen * self.rate, \
                                         1.0 / self.dataLen * self.rate],\
                     mul=self.amp).out() # Table playback. 
        self.t.replace(list(audioVec))
        self.c.setTable(self.t)
        time.sleep(self.dataLen/self.rate)
        self.c.stop()
        
    def stopAudio(self):
        self.c.stop()

In [4]:
# Create a rigid window to ramp on/off the audio vector to prevent clipping. 
# But I still hear clipping. 
def makeWindow(s, rampUp = 0.02, rampDown = 0.02):
    w = np.append(np.linspace(0, 1.0, num = int(s * rampUp)), \
      np.ones(s - int(s * rampUp)- int(s * rampDown)))
    w = np.append(w, np.linspace(1.0, 0, num = int(s * rampDown)))
    return w

# Turn list into integer
def list2int(numbers):
    return int(''.join([ "%d"%x for x in numbers]))

# Add some silence to the end
def addSilence(av, bs = 5000):
    return np.append(av, np.zeros(0.2 * bs))

# Slider update function, for the interaction
def sliderUpdate(val):
    global dt, resistant
    sigma = ssigma.val
    exp_table = updateExpTable(sigma, delta_dis)
    dt = sdt.val
    resistant = sresistant.val
    
def sigmaUpdate(val):
    global sigma, exp_table, delta_dis
    sigma = ssigma.val
    exp_table = updateExpTable(sigma, delta_dis)
    
# Create an exponential table for the lookup calculation of potential. 
def createExpTable( dim, sigma, exp_resolution = 1000000 ):
    delta_dis = np.linspace(0.0, dim, num = exp_resolution) # Range of distance difference. 
    tab = updateExpTable(sigma, delta_dis)
    return tab, dim

def updateExpTable(sigma, delta_dis):
    sigma2 = sigma * sigma
    return exp(- delta_dis/sigma2)/sigma2

In [5]:
# Currently this plotpot is only for interactive mode. 
# Eventually, data visualisation is not needed because it is aimed just for the sonification. 
# Data visualisation can be provided offline as a comparison. 
# For this method, visulisation is no use for higher dimension data anytaway 
def plotpot(data2d, fig2, sigma=.1, Nx=40, Ny=40):
    # ensure the data sending for plotpot is always in 2d. 
    potmap = zeros((Nx, Ny))
    for i in range(Nx):
        for j in range(Ny):
            x = float(i)/Nx-0.5
            y = float(j)/Ny-0.5
            potmap[j,i] = potential_ds(data2d, np.array([x,y]), sigma)
    fig2.clf()
    matshow(potmap, cmap=cm.gray, extent=(-0.5,0.5,0.5,-0.5), fignum = 2)    
    fig2.gca().plot(data[:,0], data[:,1], ".")

def initializePot(data, N):  # It takes data and number of rows. 
    sctPlot = ax.scatter(data[:,0], data[:,1],  c="blue", picker = 2, s=[50]*N)
    fig.subplots_adjust(bottom=0.2, left=0.1)
    plt.grid(False)
    plt.axis([-0.6, 0.6, -0.6, 0.6])
    return sctPlot
    
def plotpot2(data2d, fig, sigma=.1, Nx=40, Ny=40, figure_num = 1):
    # ensure the data sending for plotpot is always in 2d.    
    potmap = zeros((Nx, Ny))
    for i in range(Nx):
        for j in range(Ny):
            x = float(i)/Nx-0.5
            y = float(j)/Ny-0.5
            potmap[j,i] = potential_ds(data2d, np.array([x,y]), sigma)
    matshow(potmap, cmap=cm.gray, extent=(-0.5,0.5,0.5,-0.5), fignum = figure_num) 
    return potmap

def spectrum(av,fs):
    NFFT = 1024
    figure()
    specgram(audioVec, NFFT = NFFT, Fs = fs, noverlap = 900, cmap= cm.gist_heat)
    show()
    
def stdData(data, dim):
    for i in range(dim):
        # S1 Standardize data. 
        data[:, i] = (data[:,i] - np.mean(data[:,i]))/np.std(data[:,1])
        # limit range to -.5 ~ .5 in each dimension
        data[:,i] = data[:,i] / np.max(np.absolute(data[:,i]))
        data[:,i] = data[:,i] / 2  
    return data
    

In [7]:
%%cython
cimport numpy as np
import numpy as np
from libc.stdlib cimport rand, malloc, free
from libc.math cimport exp
from matplotlib import pyplot as plt
import time

def potential_ds(np.ndarray[np.float64_t, ndim = 2] data,\
                 np.ndarray[np.float64_t, ndim = 1] grid, double sigma=0.2):
    cdef int N, dim
    N, dim = data.shape[0], data.shape[1]
    cdef double potential, p_sum = 0.0
    cdef int j, i
    cdef double sigma2 = sigma *sigma 
    
    for j in range(N):
        for i in range(dim):
            p_sum += (grid[i] - data[j,i]) * (grid[i] - data[j,i])
            
        potential += -exp (-0.5*p_sum / sigma2)
        p_sum = 0
    return potential
    
# put pos outside and make it a np array. 
def PTSM(np.ndarray[np.float64_t, ndim = 1] pos, np.ndarray[np.float64_t, ndim = 2] X,\
         np.ndarray[np.float64_t, ndim = 1] vel, np.ndarray[np.float64_t, ndim = 1] lookupExp,\
         int table_res, double norm_max, double sigma=0.25,\
         double dt=0.01, double r=0.99, int Nsamp=5000, double compensation = 0.01 ):
    cdef int N, dim
    N, dim = X.shape[0], X.shape[1]
    cdef int i,j, steps,lookupIdx  # These 3 are for iterations. 
    cdef double sigma2,m
    cdef double d,V, v_sum = 0.0
    cdef double force1p_sum = 0.0
    cdef double vel_sum = 0.0
    cdef double *force = <double *>malloc(dim * sizeof(double))
    cdef double *trj = <double *>malloc(Nsamp * (dim+ 1) *sizeof(double))
    cdef double *velocity = <double *>malloc(dim * sizeof(double))
    cdef double *position = <double *>malloc(dim * sizeof(double))
    cdef double *temp = <double *>malloc(dim * sizeof(double)) 
    cdef double testing = 0.0

    for i in range(dim):
        velocity[i] = vel[i]
        position[i] = pos[i]
        
    sigma2 = sigma * sigma
    m = compensation/sigma2
    #--------------------
       # Force
    for steps in range(Nsamp): 
        for i in range(dim):
            force[i] = 0
        for j in range (N):
            for i in range (dim):
                temp[i] = (position[i] - X[j,i])
                force1p_sum  += temp[i] * temp[i]
            # Look for exp index 
            lookupIdx = int(force1p_sum/norm_max * table_res )
            # Prevent index become much bigger than the table 
            if (lookupIdx >= table_res):
                lookupIdx = table_res - 1
            for i in range (dim):  
#                 print lookupExp[lookupIdx]
                force[i] += -temp[i] * lookupExp[lookupIdx]
               
            force1p_sum = 0
    #------------
    #         Now update pos and vel 
    #         This is the main part to get the trajectory information in terms of the new velocity 
    #         and position
        for i in range(dim):
            # Should be dt /m * force[i], but since m = 1.0
            velocity[i] = r * velocity[i] + dt * force[i] / m 
            # This part doesn't ensure velocity to be 
            # constantly dropping. 
            # A potential solution is ot put m = k/sigma2, This compensates
            position[i] = position[i] + dt * velocity[i]
            vel_sum += velocity[i] 

            #-----------------------  
    #         # Put velocity and new position into the trj array. 
        for i in range(dim + 1):
            if i == 0: # The first column is always given to the velocity
                trj[steps * (dim + 1) + i] =  vel_sum
            else:
                trj[steps * (dim + 1) + i] = position[i - 1]
        vel_sum = 0
#     # put trj into a numpy array. Because you can't return a c-array.              
    resultTrj = np.zeros(Nsamp * (dim + 1), dtype = np.float64)
    resultVel = np.zeros(dim)
    # Move c array to numpy array for return 
    for i in range(Nsamp* (dim + 1)):
        resultTrj[i] = trj[i]     
    for i in range(dim): 
        resultVel[i] = velocity[i]  
    # Transpose matrix
    resultTrj = np.reshape(resultTrj, (-1, dim + 1))  # Correct the matrix shape. 
#     # Free up memories. 
#     print testing
    free(trj)
    free(force)
    free(velocity)
    free(position)
    return resultTrj, resultVel


/Users/jiajunyang/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/utils/path.py:264: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [26]:
# if testMode == T, a random example is used as the data. 
# else, you can load the actual data txt files. 
testMode = True
if testMode == True:
    dim = 5
    data = np.random.rand(500, dim) - np.ones(dim)* 0.5
#     data = np.random.rand(500,2)-[0.5,  0.5]
    N, dim = data.shape[0], data.shape[1]
    plot(data[:,0], data[:,1], ".")

else:
    # Load test data from file 
#     data = np.loadtxt("data/dim032.txt")
    # Still have problem loading csv file. 
    data = np.genfromtxt("data/posture.csv",delimiter=",", skip_header = 1)
    headers = np.genfromtxt("data/posture.csv", dtype = np.str_, delimiter = ",", max_rows = 1 )
    N, dim = data.shape[0], data.shape[1]
#     data = stdData(data, dim)
#     plot(data[:,0], data[:,1], ".")

# Create exp lookup
exp_resolution = 1000000 # Resolution for lookup table
delta_dis = np.linspace(0.0, dim, num = exp_resolution) # Range of distance difference. 
m_comp = 0.001 # mass compensation
norm_max = dim
sigma = 0.1
t0 = time.time()
exp_table = updateExpTable(sigma, delta_dis)
t1 = time.time()
print (t1 - t0)

0.0247180461884


## Create loop up table for exp()


## Interaction here. 

Offer visualise mode and only sound mode. 


In [28]:
%pylab
fig, ax = plt.subplots()
# Init paras
m_comp = 0.001 # mass compensation
dt = 0.005
r = 0.9995
# r = 1.0
t = 1.0 # Time in second per piece
resistant = 0.9995
# resistant = 1.0
blockSize = 5000   # Buffer size for trajectory
audioVecSize  = t *  blockSize  # I define that 5000 steps will return 1 second of audio 
exp_table, norm_max = createExpTable(dim, sigma, exp_resolution = 10000000)
window = makeWindow(audioVecSize, rampUp = 0.02, rampDown = 0.02) # Windowing for audio
# Initilize audio
soundEngine = playAudioNoThread(fs, dataLen = t, play_back_rate = 1, bs=audioVecSize, amp = 0.9)
#-------------------------------------------------#

# Initialize the plot windows. 
sctPlot = initializePot(data, N)

def on_pick(event): 
    global data, sigma, resultWindow, vel, resistant, window, audioVecSize, fs, t,\
    t_exp, r2, t_res, norm_max, blockSize
    t0 = time.time()
    drawResult = True
    # Generate a random velocity at each trigger. 
    # Need to know whether this is correct. 
    vel = rand(dim); vel /= -norm(vel)/1
    artist = event.artist # Current click event
    ind = np.array(event.ind) # Get the index of the clicked data
    # To prevent retriving multiple points. Always take the first index for PTSM
    pos = np.array(data[ind[0],:])
    # Get PTSM trajectory information
    trj, junk = PTSM(pos, data, vel, exp_table, exp_resolution, \
                norm_max, sigma = sigma, dt = dt, r = resistant,\
                Nsamp = audioVecSize, compensation = m_comp)
    audioVec = trj[:,0] / np.max(np.absolute(trj[:,0])) * window
#     audioVec = addSilence(audioVec, blockSize) # Now add silence will cause the differ in table size. 
    soundEngine.single(audioVec)
    
    if drawResult == True:
        plotpot(data[:, 0:2], fig2 = resultWindow,  sigma = sigma )
        resultWindow.gca().plot(trj[:,1], trj[:,2], "-", lw=0.7, c = "green")
        # Mark the beginning and the end of trajectory . 
        resultWindow.gca().plot(trj[0,1], trj[0,2], "o",  c = "yellow")
        resultWindow.gca().plot(trj[audioVecSize - 1,1], trj[audioVecSize - 1,2], "x",  c = "red")
        resultWindow.gca().axis([-0.6, 0.6, -0.6, 0.6])
        resultWindow.canvas.draw()
    print time.time() - t0

# ---------------#
# Create a slider for setting up the velocity
axcolor = 'lightgoldenrodyellow'
# Create sliders for sigma and dt. 
axSigma = plt.axes([0.1, 0.1, 0.8, 0.02], axisbg = axcolor)
axDt = plt.axes([0.1, 0.06, 0.8, 0.02], axisbg = axcolor)
axR = plt.axes([0.1, 0.02, 0.8, 0.02], axisbg = axcolor)

ssigma = Slider(axSigma, "Sigma", 0.001, 0.15, valinit=0.06, color='blue')
sdt = Slider(axDt, "dt", 0.001, 0.02, valinit = 0.01, color = 'blue')
sresistant = Slider(axR, "r", 0.999, 1.0, valinit = resistant, color = 'blue')

ssigma.on_changed(sigmaUpdate)
sdt.on_changed(sliderUpdate)
sresistant.on_changed(sliderUpdate)
resultWindow = figure(2, figsize=(12, 12))
fig.canvas.mpl_connect('pick_event', on_pick)




Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


15

In [9]:
norm_max

2